In [1]:
import os
import logging
from datetime import datetime

In [2]:


import boto3
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\hemch\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [34]:
region = sagemaker.Session().boto_region_name
print(region)

us-east-1


In [35]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import json
import logging
import sys
from pathlib import Path

import ipytest

#CODE_FOLDER = Path("code")
#sys.path.extend([f"./{CODE_FOLDER}"])

#DATA_FILEPATH = "../data/penguins.csv"

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [36]:
import os

bucket = os.environ["BUCKET"]+'/sagemaker-course'
role = os.environ["ROLE"]

COMET_API_KEY = os.environ.get("COMET_API_KEY", None)
COMET_PROJECT_NAME = os.environ.get("COMET_PROJECT_NAME", None)

In [37]:
bucket

'beercafe-ml-bucket/sagemaker-course'

In [38]:
from pathlib import Path
DATA_LOC = Path("../data/")
print(DATA_LOC)

..\data


In [39]:
job_name_base = 'census-data-processor'

# Processing data with inbuilt sklearn container

## Create Processor

In [47]:
from sagemaker.session import Session
sagemaker_session=Session(default_bucket='beercafe-learn-sagemaker', default_bucket_prefix='census-processing-op')

In [41]:
from sagemaker.sklearn.processing import SKLearnProcessor


sklearn_processor = SKLearnProcessor(
    framework_version="1.0-1",
    role=role,
    instance_type='ml.m5.xlarge',
    instance_count=1,
    volume_size_in_gb=5,
    base_job_name=job_name_base,
    sagemaker_session=sagemaker_session
)

## Download data from sagemaker itself

In [42]:
import pandas as pd

DATA_LOC_CENSUS = DATA_LOC / 'census'
print(DATA_LOC_CENSUS)

s3 = boto3.client("s3")
s3.download_file(
    "sagemaker-sample-data-{}".format(region),
    "processing/census/census-income.csv",
    f"{DATA_LOC_CENSUS}/census-income.csv",
)
df = pd.read_csv(f"{DATA_LOC_CENSUS}/census-income.csv")
df.to_csv(f"{DATA_LOC_CENSUS}/dataset.csv")
df.head()

..\data\census


,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,...,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year,income
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.


## Prepare Processing script

In [32]:
CODE_FOLDER = Path('../code/census')
print(CODE_FOLDER)

..\code\census


In [28]:
%%writefile preprocessing.py
import pandas as pd
import os
from sklearn.model_selection import train_test_split

input_data_path = os.path.join("/opt/ml/processing/input", "dataset.csv")
df = pd.read_csv(input_data_path)
print("Shape of data is:", df.shape)
train, test = train_test_split(df, test_size=0.2)
train, validation = train_test_split(train, test_size=0.2)

try:
    os.makedirs("/opt/ml/processing/output/train")
    os.makedirs("/opt/ml/processing/output/validation")
    os.makedirs("/opt/ml/processing/output/test")
    print("Successfully created directories")
except Exception as e:
    # if the Processing call already creates these directories (or directory otherwise cannot be created)
    print(e)
    print("Could not make directories")
    pass

try:
    train.to_csv("/opt/ml/processing/output/train/train.csv")
    validation.to_csv("/opt/ml/processing/output/validation/validation.csv")
    test.to_csv("/opt/ml/processing/output/test/test.csv")
    print("Wrote files successfully")
except Exception as e:
    print("Failed to write the files")
    print(e)
    pass

print("Completed running the processing job")

Overwriting preprocessing.py


## Run Processing job

In [43]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code="../code/census/preprocessing.py",
    inputs=[ProcessingInput(source='../data/census/dataset.csv',destination='/opt/ml/processing/input')],
    outputs=[ProcessingOutput(source='/opt/ml/processing/output/train'),
        ProcessingOutput(source='/opt/ml/processing/output/validation'),
        ProcessingOutput(source='/opt/ml/processing/output/test')]
)

.........................Dummy file



In [44]:
import boto3


s3_client = boto3.client("s3")
default_bucket = sagemaker.Session().default_bucket()

In [45]:
default_bucket

'sagemaker-us-east-1-767398043272'